# Transfer Learning with MobileNetV3

<a name='1'></a>
## 1 - Packages

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## MobileNetV3


In [3]:
IMG_SIZE = (512, 512)
IMG_SHAPE = IMG_SIZE + (3,)

mobile_v3_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(512,512,3),
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights=None,
    input_tensor=None,
    classes=1000,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation='softmax',
    include_preprocessing=True
)

In [4]:
print("Number of layers in the base model: ", len(mobile_v3_model.layers))

Number of layers in the base model:  263


In [5]:
mobile_v3_model.summary()

Model: "MobilenetV3large"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 256, 256, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                   

In [23]:
[f'{layer.output_shape}  |  {layer.count_params(): 9,d}  |  {layer.name}' for layer in mobile_v3_model.layers[-10: ]]

['(None, 16, 16, 960)  |          0  |  expanded_conv_14/squeeze_excite/Mul',
 '(None, 16, 16, 160)  |    153,600  |  expanded_conv_14/project',
 '(None, 16, 16, 160)  |        640  |  expanded_conv_14/project/BatchNorm',
 '(None, 16, 16, 160)  |          0  |  expanded_conv_14/Add',
 '(None, 16, 16, 960)  |    153,600  |  Conv_1',
 '(None, 16, 16, 960)  |      3,840  |  Conv_1/BatchNorm',
 '(None, 16, 16, 960)  |          0  |  tf.__operators__.add_27',
 '(None, 16, 16, 960)  |          0  |  re_lu_38',
 '(None, 16, 16, 960)  |          0  |  tf.math.multiply_27',
 '(None, 16, 16, 960)  |          0  |  multiply_19']

# DataSet

In [ ]:
dataset, info = tfds.load(
    'coco/2017',
    split='train',
    batch_size=32,
    data_dir=r'D:\tensorflow_datasets',
    download=True,
    shuffle_files=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

In [ ]:
tfds.show_examples(dataset, info)